Exercise pose matching algorithm

This notebook runs through the exercise pose matching algorithm for PAVE.

# Initiate ExPose

In [ ]:
%cd /content
!rm -rf /content/expose
!git clone https://github.com/vchoutas/expose
%cd expose
#!pip install -r requirements.txt

/content
Cloning into 'expose'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 138 (delta 25), reused 19 (delta 19), pack-reused 100
Receiving objects: 100% (138/138), 6.70 MiB | 33.32 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/expose


In [ ]:
import os
import shutil
from google.colab import drive
drive.mount('/content/gdrive')
req_file = "requirementsv3.txt"
gdrive_path = "/content/gdrive/MyDrive/datasets"
dst_path = "/content/expose"
shutil.copyfile(os.path.join(gdrive_path, req_file), os.path.join(dst_path, req_file))

!pip install -r requirementsv3.txt

Mounted at /content/gdrive
     |████████████████████████████████| 50 kB 4.2 MB/s 
     |████████████████████████████████| 58 kB 7.3 MB/s 
     |████████████████████████████████| 11.2 MB 43.9 MB/s 
     |████████████████████████████████| 4.6 MB 66.4 MB/s 
     |████████████████████████████████| 3.1 MB 80.1 MB/s 
     |████████████████████████████████| 1.2 MB 86.8 MB/s 
     |████████████████████████████████| 646 kB 86.4 MB/s 
     |████████████████████████████████| 596 kB 89.5 MB/s 
     |████████████████████████████████| 930 kB 90.7 MB/s 
     |████████████████████████████████| 2.2 MB 87.0 MB/s 
     |████████████████████████████████| 978 kB 54.9 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=930c465284ac37c80ac92fefb66ec72aa6639e278bc9dd3ffa3e315c60c0479b
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.w

Download expose_data.zip from https://expose.is.tue.mpg.de/

In [ ]:
%cd /content/
from google.colab import drive
drive.mount('/content/gdrive')

EXPOSE_PATH = '/content/gdrive/MyDrive/datasets/expose_data.zip' # @param {type:"string"}

DATA_PATH = '/content/expose'
!unzip -n '$EXPOSE_PATH' -d  $DATA_PATH

/content
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Archive:  /content/gdrive/MyDrive/datasets/expose_data.zip
  inflating: /content/expose/data/all_means.pkl  
   creating: /content/expose/data/checkpoints/
  inflating: /content/expose/data/checkpoints/model.ckpt  
 extracting: /content/expose/data/checkpoints/latest_checkpoint  
  inflating: /content/expose/data/conf.yaml  
  inflating: /content/expose/data/shape_mean.npy  
  inflating: /content/expose/data/SMPLX_to_J14.pkl  


Download models_smplx_v1.1.zip from https://smpl-x.is.tue.mpg.de/index.html

In [ ]:
SMPLX_ZIP_PATH = '/content/gdrive/MyDrive/datasets/models_smplx_v1_1.zip' # @param {type:"string"}


SMPLX_MODEL_PATH = '/content/expose/data'
!unzip -n '$SMPLX_ZIP_PATH' -d  $SMPLX_MODEL_PATH

Archive:  /content/gdrive/MyDrive/datasets/models_smplx_v1_1.zip
   creating: /content/expose/data/models/
   creating: /content/expose/data/models/smplx/
  inflating: /content/expose/data/models/smplx/SMPLX_NEUTRAL.npz  
 extracting: /content/expose/data/models/smplx/version.txt  
  inflating: /content/expose/data/models/smplx/SMPLX_MALE.pkl  
  inflating: /content/expose/data/models/smplx/SMPLX_FEMALE.npz  
 extracting: /content/expose/data/models/smplx/smplx_npz.zip  
  inflating: /content/expose/data/models/smplx/SMPLX_MALE.npz  
  inflating: /content/expose/data/models/smplx/SMPLX_FEMALE.pkl  
  inflating: /content/expose/data/models/smplx/SMPLX_NEUTRAL.pkl  


In [ ]:
!rm -rf /content/expose/data/expose_results
%cd /content/expose

!python demo.py --image-folder samples \
    --exp-cfg data/conf.yaml \
    --show=False \
    --output-folder /content/expose/data/expose_results \
    --save-params=False \
    --save-vis=True \
    --save-mesh=False \
    --degrees=90

/content/expose
Downloading: "https://download.pytorch.org/models/keypointrcnn_resnet50_fpn_coco-fc266e95.pth" to /root/.cache/torch/hub/checkpoints/keypointrcnn_resnet50_fpn_coco-fc266e95.pth
100% 226M/226M [00:03<00:00, 69.8MB/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Processing with R-CNN: 100% 1/1 [00:02<00:00,  2.26s/it]
2022-05-21 06:29:43.430 | INFO     | __main__:main:241 - Saving results to: /content/expose/data/expose_results
2022-05-21 06:29:43.434 | WARNING  | expose.models.attention.predictor:__init__:92 - Apply hand network on body: True
2022-05-21 06:29:43.435 |

# Run Openpose and ExPose to get rotated SMPL model

OPENPOSE

Install OPENPOSE to extract keypoints

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

Selecting previously unselected package libgflags2.2.
(Reading database ... 155629 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

Cut video and extract keypoints

In [ ]:
# create folder
!rm -rf /content/expose/folder
!mkdir /content/expose/folder
!mkdir /content/expose/folder/keypoints
!mkdir /content/expose/folder/images
%cd /content/expose

/content/expose


Extract image frame by frame from video

In [ ]:
import cv2
import os
import shutil

images = {}

# length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# print(length)

#!mkdir /content/expose/openpose/output_images
#vid_file = "person2_ex1_NPr_P.mp4"
vid_file = "withSpec_pain_incorrectpose.mp4"
expose_file = "inferenceV2.py"
gdrive_path = "/content/gdrive/MyDrive/datasets"
dst_path = "/content/expose"

shutil.copyfile(os.path.join(gdrive_path, vid_file), os.path.join(dst_path, vid_file))
shutil.copyfile(os.path.join(gdrive_path, expose_file), os.path.join(dst_path, expose_file))


cap = cv2.VideoCapture('withSpec_pain_incorrectpose.mp4')
i = 0
#%cd /content/expose/openpose/output_images
%cd /content/expose/folder/images
fps = 30
time = 15
cropval = 0
while(cap.isOpened()):
    ret, frame = cap.read()
     
    # This condition prevents from infinite looping
    # incase video ends.
    if ret == False:
        break
    frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)
    
    # Crop frame
    #print(frame.shape)
    imgheight = frame.shape[0]
    imgwidth = frame.shape[1]
    frame = frame[int(imgheight*cropval/2):imgheight-int(imgheight*cropval/2), int(imgwidth*cropval/2):imgwidth-int(imgwidth*cropval/2),:] 

    # Save Frame by Frame into disk using imwrite method
    cv2.imwrite('video1_'+str(i).zfill(12)+'.jpg', frame)
    
    i += 1

    # if i >= (time*fps):
    #     break

cap.release()
cv2.destroyAllWindows()

%cd /content/expose/

/content/expose/folder/images
/content/expose


In [ ]:
!rm -rf youtube.mp4
!rm -rf output1.mp4
!rm -rf video1.mp4
# download the youtube with the given ID
#!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
# cut the first 5 seconds
# person2_ex4_NPr_P
#!ffmpeg -y -loglevel info -i youtube.mp4 -t 5 video.mp4
!ffmpeg -y -loglevel info -i withSpec_pain_incorrectpose.mp4 video1.mp4
#!ffmpeg -y -loglevel info -i /content/gdrive/MyDrive/datasets/person2_ex4_NPr_P.mp4 -t 5 video1.mp4
# detect poses on the these 5 seconds
!rm -rf openpose.avi
#!cd openpose && ./build/examples/openpose/openpose.bin --video ../video1.mp4 --write_json ./output/ --display 0  -write_images ./output_images/ --write_images_format jpg --write_video ../openpose.avi
#!cd openpose && ./build/examples/openpose/openpose.bin --video ../video1.mp4 --face --hand --write_json /content/expose/folder/keypoints/ --display 0  --render_pose 0  
#!cd openpose && ./build/examples/openpose/openpose.bin --video ../video1.mp4 --face --hand --write_json /content/expose/folder/keypoints/ --display 0 --write_video ../openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --image_dir ../folder/images --face --hand --write_json /content/expose/folder/keypoints/ --display 0 --write_video ../openpose.avi --write_video_fps 30
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output1.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

Run EXPOSE on extracted keypoints and image frames

In [ ]:
#!rm -rf /content/gdrive/MyDrive/ISS/folder/output
!rm -rf /content/expose/data/expose_results1
%cd /content/expose

!python inferenceV2.py --exp-cfg data/conf.yaml \
           --datasets openpose \
           --exp-opts datasets.body.batch_size 4 datasets.body.openpose.data_folder folder \
           --show=False \
           --output-folder /content/expose/data/expose_results1 \
           --save-params=False \
           --save-vis=True \
           --save-mesh=False \
           --degrees 0 45 90 135 180 225 270 315




/content/expose
2022-05-21 06:48:28.645 | INFO     | __main__:main:132 - Saving results to: /content/expose/data/expose_results1
2022-05-21 06:48:28.654 | WARNING  | expose.models.attention.predictor:__init__:92 - Apply hand network on body: True
2022-05-21 06:48:28.654 | WARNING  | expose.models.attention.predictor:__init__:94 - Apply hand network on hands: True
2022-05-21 06:48:28.654 | WARNING  | expose.models.attention.predictor:__init__:95 - Predict hands: True
2022-05-21 06:48:28.655 | WARNING  | expose.models.attention.predictor:__init__:102 - Predict head: True
2022-05-21 06:48:28.655 | INFO     | expose.models.attention.predictor:__init__:109 - Condition hand on body: True
2022-05-21 06:48:28.655 | INFO     | expose.models.attention.predictor:__init__:113 - Condition hand wrist pose on body: True
2022-05-21 06:48:28.655 | INFO     | expose.models.attention.predictor:__init__:118 - Condition hand finger pose on body: True
2022-05-21 06:48:28.655 | INFO     | expose.models.atten

Reorganise folders and SMPL images

In [ ]:
%cd /content/expose/data/

!rm -rf /content/expose/data/expose_results2
!mkdir /content/expose/data/expose_results2

#save_path = "/content/expose/data/expose_results2"

!rm -rf /content/expose/data/expose_results000
!mkdir /content/expose/data/expose_results000

!rm -rf /content/expose/data/expose_results045
!mkdir /content/expose/data/expose_results045

!rm -rf /content/expose/data/expose_results135
!mkdir /content/expose/data/expose_results135

!rm -rf /content/expose/data/expose_results225
!mkdir /content/expose/data/expose_results225

!rm -rf /content/expose/data/expose_results315
!mkdir /content/expose/data/expose_results315

!rm -rf /content/expose/data/expose_results225f
!mkdir /content/expose/data/expose_results225f

!rm -rf /content/expose/data/expose_results315f
!mkdir /content/expose/data/expose_results315f

save_path0 = "/content/expose/data/expose_results000"
save_path1 = "/content/expose/data/expose_results045"
save_path2 = "/content/expose/data/expose_results135"
save_path3 = "/content/expose/data/expose_results225"
save_path4 = "/content/expose/data/expose_results315"
save_path5 = "/content/expose/data/expose_results225f"
save_path6 = "/content/expose/data/expose_results315f"

import os
import shutil

# assign directory
directory = 'expose_results1'
 
# iterate over files in
# that directory
for foldername in os.listdir(directory):
    fol = os.path.join(directory, foldername)
    for filename in os.listdir(fol):
      if filename == "hd_rendering_045.png":
        filename00 = foldername[:-4] + "_" + filename[-7:]
        shutil.copyfile(os.path.join(fol, filename), os.path.join(save_path1, filename00))
    
      elif filename == "hd_rendering_135.png":
        filename90 = foldername[:-4] + "_" + filename[-7:]
        shutil.copyfile(os.path.join(fol, filename), os.path.join(save_path2, filename90))
      
      elif filename == "hd_rendering_000.png":
        filename0 = foldername[:-4] + "_" + filename[-7:]
        shutil.copyfile(os.path.join(fol, filename), os.path.join(save_path0, filename0))
      
      elif filename == "hd_rendering_225.png":
        filename180 = foldername[:-4] + "_" + filename[-7:]
        shutil.copyfile(os.path.join(fol, filename), os.path.join(save_path3, filename180))
        filename180f = foldername[:-4] + "_f" + filename[-7:]
        src = cv2.imread(os.path.join(fol, filename))
        image = cv2.flip(src, 1)
        os.chdir(save_path5)
        cv2.imwrite(filename180f, image)
        os.chdir("/content/expose/data/")
      
      elif filename == "hd_rendering_315.png":
        filename270 = foldername[:-4] + "_" + filename[-7:]
        shutil.copyfile(os.path.join(fol, filename), os.path.join(save_path4, filename270))
        filename270f = foldername[:-4] + "_f" + filename[-7:]
        src = cv2.imread(os.path.join(fol, filename))
        image = cv2.flip(src, 1)
        os.chdir(save_path6)
        cv2.imwrite(filename270f, image)
        os.chdir("/content/expose/data/")
        

      # elif filename == "hd_rendering_135.png":
      #   filename135 = foldername[:-4] + "_" + filename[-7:]
      #   shutil.copyfile(os.path.join(fol, filename), os.path.join(save_path, filename135))
      
%cd /content/expose

/content/expose/data
/content/expose


# Run Openpose on rotated ExPose SMPL models

Run openpose on rotated Expose SMPL models

In [ ]:
%cd /content/expose

!rm -rf /content/expose/data/expose_keypoints045
!mkdir /content/expose/data/expose_keypoints045

!rm -rf openpose045.avi
!rm -rf output045.mp4

!cd openpose && ./build/examples/openpose/openpose.bin  --image_dir ../data/expose_results045 --write_json /content/expose/data/expose_keypoints045/  --display 0 --write_video ../openpose045.avi --write_video_fps 30

!ffmpeg -y -loglevel info -i openpose045.avi output045.mp4

/content/expose
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 60.299766 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshi

In [ ]:
%cd /content/expose

!rm -rf /content/expose/data/expose_keypoints135
!mkdir /content/expose/data/expose_keypoints135

!rm -rf openpose135.avi
!rm -rf output135.mp4

!cd openpose && ./build/examples/openpose/openpose.bin  --image_dir ../data/expose_results135 --write_json /content/expose/data/expose_keypoints135/  --display 0 --write_video ../openpose135.avi --write_video_fps 30

!ffmpeg -y -loglevel info -i openpose135.avi output135.mp4

/content/expose
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 55.613226 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshi

In [ ]:
%cd /content/expose

!rm -rf /content/expose/data/expose_keypoints225
!mkdir /content/expose/data/expose_keypoints225

!rm -rf openpose225.avi
!rm -rf output225.mp4

!cd openpose && ./build/examples/openpose/openpose.bin  --image_dir ../data/expose_results225 --write_json /content/expose/data/expose_keypoints225/  --display 0 --write_video ../openpose225.avi --write_video_fps 30

!ffmpeg -y -loglevel info -i openpose225.avi output225.mp4

/content/expose
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 56.095219 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshi

In [ ]:
%cd /content/expose

!rm -rf /content/expose/data/expose_keypoints315
!mkdir /content/expose/data/expose_keypoints315

!rm -rf openpose315.avi
!rm -rf output315.mp4

!cd openpose && ./build/examples/openpose/openpose.bin  --image_dir ../data/expose_results315 --write_json /content/expose/data/expose_keypoints315/  --display 0 --write_video ../openpose315.avi --write_video_fps 30

!ffmpeg -y -loglevel info -i openpose315.avi output315.mp4

/content/expose
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 57.006644 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshi

In [ ]:
%cd /content/expose

!rm -rf /content/expose/data/expose_keypoints225f
!mkdir /content/expose/data/expose_keypoints225f

!rm -rf openpose225f.avi
!rm -rf output225f.mp4

!cd openpose && ./build/examples/openpose/openpose.bin  --image_dir ../data/expose_results225f --write_json /content/expose/data/expose_keypoints225f/  --display 0 --write_video ../openpose225f.avi --write_video_fps 30

!ffmpeg -y -loglevel info -i openpose225f.avi output225f.mp4

/content/expose
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 50.414091 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshi

In [ ]:
%cd /content/expose

!rm -rf /content/expose/data/expose_keypoints315f
!mkdir /content/expose/data/expose_keypoints315f

!rm -rf openpose315f.avi
!rm -rf output315f.mp4

!cd openpose && ./build/examples/openpose/openpose.bin  --image_dir ../data/expose_results315f --write_json /content/expose/data/expose_keypoints315f/  --display 0 --write_video ../openpose315f.avi --write_video_fps 30

!ffmpeg -y -loglevel info -i openpose315f.avi output315f.mp4

/content/expose
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 50.315097 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshi

# Multi-view 3D keypoint generation

Triangulation for multi-view pose estimation (3D coordinates)

In [ ]:
import os
import json

folderlist1 = sorted(os.listdir("/content/expose/data/expose_keypoints045"))
folderlist2 = sorted(os.listdir("/content/expose/data/expose_keypoints135"))
folderlist3 = sorted(os.listdir("/content/expose/data/expose_keypoints225f"))
folderlist4 = sorted(os.listdir("/content/expose/data/expose_keypoints315f"))
print(folderlist1)
print(folderlist2)
print(folderlist3)
print(folderlist4)

path1 = "/content/expose/data/expose_keypoints045"
path2 = "/content/expose/data/expose_keypoints135"
path3 = "/content/expose/data/expose_keypoints225f"
path4 = "/content/expose/data/expose_keypoints315f"

data1list = []
data2list = []
data3list = []
data4list = []

for j in range(0,len(folderlist1)):

  # Opening JSON file
  with open(os.path.join(path1,folderlist1[j])) as json_file1:
      data1 = json.load(json_file1)
  with open(os.path.join(path2,folderlist2[j])) as json_file2:
      data2 = json.load(json_file2)
  with open(os.path.join(path3,folderlist3[j])) as json_file3:
      data3 = json.load(json_file3)
  with open(os.path.join(path4,folderlist4[j])) as json_file4:
      data4 = json.load(json_file4)
  # print("Frame:"+str(j))
  # print(data1['people'][0]['pose_keypoints_2d'])
  # print(data2['people'][0]['pose_keypoints_2d'])
  # print(data3['people'][0]['pose_keypoints_2d'])
  # print(data4['people'][0]['pose_keypoints_2d'])
  data1list.append(data1['people'][0]['pose_keypoints_2d'])
  data2list.append(data2['people'][0]['pose_keypoints_2d'])
  data3list.append(data3['people'][0]['pose_keypoints_2d'])
  data4list.append(data4['people'][0]['pose_keypoints_2d'])

print(data1list[-1])
print(data2list[-1])
print(data3list[-1])
print(data4list[-1])

['video1_000000000000_045_keypoints.json', 'video1_000000000001_045_keypoints.json', 'video1_000000000002_045_keypoints.json', 'video1_000000000003_045_keypoints.json', 'video1_000000000004_045_keypoints.json', 'video1_000000000005_045_keypoints.json', 'video1_000000000006_045_keypoints.json', 'video1_000000000007_045_keypoints.json', 'video1_000000000008_045_keypoints.json', 'video1_000000000009_045_keypoints.json', 'video1_000000000010_045_keypoints.json', 'video1_000000000011_045_keypoints.json', 'video1_000000000012_045_keypoints.json', 'video1_000000000013_045_keypoints.json', 'video1_000000000014_045_keypoints.json', 'video1_000000000015_045_keypoints.json', 'video1_000000000016_045_keypoints.json', 'video1_000000000017_045_keypoints.json', 'video1_000000000018_045_keypoints.json', 'video1_000000000019_045_keypoints.json', 'video1_000000000020_045_keypoints.json', 'video1_000000000021_045_keypoints.json', 'video1_000000000022_045_keypoints.json', 'video1_000000000023_045_keypoint

In [ ]:
import numpy as np
import cv2


def calibrateimgflip(keypoint1,keypoint2,numkeypoints,thrshld):
  a3xN = np.zeros([3,numkeypoints])
  b3xN = np.zeros([3,numkeypoints])
  c3xN = np.zeros([3,numkeypoints])

  i2 = [0,1,5,6,7,2,3,4,8,12,13,14,9,10,11]
  for i,v in enumerate(range(0,numkeypoints*3,3)):
    a3xN[0,i] = keypoint1[v]
    a3xN[1,i] = keypoint1[v+1]
    a3xN[2,i] = 1.0
    b3xN[0,i2[i]] = keypoint2[v]+20
    b3xN[1,i2[i]] = keypoint2[v+1]
    b3xN[2,i2[i]] = 1.0

    if keypoint1[v+2] < thrshld:
      a3xN[2,i] = 0.0
    if keypoint2[v+2] < thrshld:
      b3xN[2,i2[i]] = 0.0

  for i,v in enumerate(range(0,numkeypoints*3,3)):
    if (a3xN[2,i] == 1.0) and (b3xN[2,i] == 1.0):
      c3xN[0,i] = np.mean([a3xN[0,i],b3xN[0,i]])
      c3xN[1,i] = np.mean([a3xN[1,i],b3xN[1,i]])
      c3xN[2,i] = 1.0
    elif a3xN[2,i] == 1.0:
      c3xN[0,i] = np.mean(a3xN[0,i])
      c3xN[1,i] = np.mean(a3xN[1,i])
      c3xN[2,i] = 1.0
      b3xN[2,i] = 1.0
    elif b3xN[2,i] == 1.0:
      c3xN[0,i] = np.mean(b3xN[0,i])
      c3xN[1,i] = np.mean(b3xN[1,i])
      c3xN[2,i] = 1.0
      a3xN[2,i] = 1.0
    else:
      c3xN[0,i] = 0.0
      c3xN[1,i] = 0.0
      c3xN[2,i] = 1.0
      a3xN[2,i] = 1.0
      b3xN[2,i] = 1.0
 
  #print(a3xN)
  #print(b3xN)
  #print(c3xN)
  #print("--------------------------------------")

  return a3xN, b3xN, c3xN
   


In [ ]:
numkeypoints=15
thrshld = 0.5

zang = 45

gr = np.zeros([len(data1list),3,numkeypoints])
for kp,keypoint in enumerate(data1list):
  keypoint1 = data1list[kp]
  keypoint2 = data3list[kp]
  keypoint3 = data2list[kp]
  keypoint4 = data4list[kp]

  a,b,c = calibrateimgflip(keypoint1,keypoint2,numkeypoints,thrshld)
  d,e,f = calibrateimgflip(keypoint3,keypoint4,numkeypoints,thrshld)
  g = np.zeros([3,numkeypoints])
  g[:2,:] = c[:2,:]
  g[2,:] = f[0,:] 

  # rotate all points by 45deg (Yaxis)
  zang = np.deg2rad(zang)
  zrotmat = np.array([[np.cos(zang),0,np.sin(zang)],
                      [0,1,0],
                      [-np.sin(zang),0,np.cos(zang)]])

  gr[kp,:,:] = np.dot(zrotmat,g)

print(gr.shape)

(1442, 3, 15)


In [ ]:
import math
def calcAngles(P1,P2):  
  x1,y1,z1 = P1
  x2,y2,z2 = P2
  distance = math.sqrt((x2-x1)**2+(y2-y1)**2+(z2 -z1)**2)
  xdiff = x2-x1
  ydiff = y2-y1
  zdiff = z2-z1
  elev = math.degrees(math.asin((z2-z1)/distance))
  azimuth = math.degrees(math.atan2((x2-x1),(y2-y1)))
  return elev,azimuth,xdiff,ydiff,zdiff

In [ ]:
# define joint angles

def calbodyangle(gr,numkeypoints):
  bodyanglesmat = np.zeros([gr.shape[0],5,numkeypoints-1])
  for j,gr1 in enumerate(gr):
    bodyangles = np.zeros([5,numkeypoints-1])

    # 0-1 neck
    # 2-1 right shoulder
    # 3-2 right upper arm
    # 4-3 right lower arm
    # 5-1 left shoulder
    # 6-5 left upper arm
    # 7-6 left lower arm
    # 1-8 body
    # 9-8 right hip
    # 10-9 right upper leg
    # 11-10 right lower leg
    # 12-8 left hip
    # 13-12 left upper leg
    # 14-13 left lower leg

    jointcoord = [[1,0],[1,2],[2,3],[3,4],[1,5],[5,6],[6,7],[8,1],[8,9],[9,10],[10,11],[8,12],[12,13],[13,14]]

    for i,jc in enumerate(jointcoord):
      P1 = [gr1[0,jc[0]],gr1[2,jc[0]],-gr1[1,jc[0]]]
      P2 = [gr1[0,jc[1]],gr1[2,jc[1]],-gr1[1,jc[1]]]
      elev,azimuth,xdiff,ydiff,zdiff = calcAngles(P1,P2)
      bodyangles[0,i] = elev
      bodyangles[1,i] = azimuth
      bodyangles[2,i] = xdiff
      bodyangles[3,i] = ydiff
      bodyangles[4,i] = zdiff
    bodyanglesmat[j,:,:] = bodyangles
  return bodyanglesmat

In [ ]:
bodymat = calbodyangle(gr,15)
print(bodymat.shape)

(1442, 5, 14)


Export out correct reference data

In [ ]:
np.save('refvid3_full_t3.npy', bodymat)

In [ ]:
ref_file = "refvid3_full_t3.npy"
gdrive_path = "/content/gdrive/MyDrive/datasets"
src_path = "/content/expose"
shutil.copyfile(os.path.join(src_path, ref_file), os.path.join(gdrive_path, ref_file))

'/content/gdrive/MyDrive/datasets/refvid3_full_t3.npy'

In [ ]:
np.save('testvid3_full_t7.npy', bodymat)

In [ ]:
ref_file = "testvid3_full_t7.npy"
gdrive_path = "/content/gdrive/MyDrive/datasets"
src_path = "/content/expose"
shutil.copyfile(os.path.join(src_path, ref_file), os.path.join(gdrive_path, ref_file))

'/content/gdrive/MyDrive/datasets/testvid3_full_t7.npy'

Import correct reference data

In [ ]:
ref_file = "refvid3_full_t5.npy"
gdrive_path = "/content/gdrive/MyDrive/datasets"
dst_path = "/content/expose"
shutil.copyfile(os.path.join(gdrive_path, ref_file), os.path.join(dst_path, ref_file))

'/content/expose/refvid3_full_t5.npy'

In [ ]:
refbodymat = np.load('refvid3_full_t5.npy')
print(refbodymat.shape)

(1229, 5, 14)


In [ ]:
ref_file = "testvid3_full_t5.npy"
gdrive_path = "/content/gdrive/MyDrive/datasets"
dst_path = "/content/expose"
shutil.copyfile(os.path.join(gdrive_path, ref_file), os.path.join(dst_path, ref_file))

'/content/expose/testvid3_full_t5.npy'

In [ ]:
testbodymat = np.load('testvid3_full_t5.npy')
print(testbodymat.shape)

(1442, 5, 14)


In [ ]:
print(testbodymat[20,4,7])
print(refbodymat[20,4,7])

374.6055
385.0335


# Joint angle comparison

Compare DTW

In [ ]:
!pip install tslearn

     |████████████████████████████████| 793 kB 7.2 MB/s 


In [ ]:
import tslearn.metrics as tsl
pathlist = []
distlist = []


    # 0-1 neck
    # 2-1 right shoulder
    # 3-2 right upper arm
    # 4-3 right lower arm
    # 5-1 left shoulder
    # 6-5 left upper arm
    # 7-6 left lower arm
    # 1-8 body
    # 9-8 right hip
    # 10-9 right upper leg
    # 11-10 right lower leg
    # 12-8 left hip
    # 13-12 left upper leg
    # 14-13 left lower leg


## Whole body
# testbodymat1 = np.reshape(testbodymat[:,0:2,:],(testbodymat.shape[0],-1)).transpose(0,1)
# refbodymat1 = np.reshape(refbodymat[:,0:2,:],(refbodymat.shape[0],-1)).transpose(0,1)

# path, dist = tsl.dtw_path(testbodymat1[:,:], refbodymat1[:,:])

##Individual joint
# testbodymat1 = testbodymat[:,0:2,:]
# refbodymat1 = refbodymat[:,0:2,:]

# for i in range(0,testbodymat1.shape[2]):
#   path, dist = tsl.dtw_path(testbodymat1[:,:,i], refbodymat1[:,:,i])
#   pathlist.append(path)
#   distlist.append(dist)

##Group hands, body, legs
testbodymat1 = testbodymat[:,0:2,(2,3,5,6)]
refbodymat1 = refbodymat[:,0:2,(2,3,5,6)]

testbodymat2 = testbodymat[:,0:2,(0,1,4,7,8,11)]
refbodymat2 = refbodymat[:,0:2,(0,1,4,7,8,11)]

testbodymat3 = testbodymat[:,0:2,(9,10,12,13)]
refbodymat3 = refbodymat[:,0:2,(9,10,12,13)]

testbodymat1 = np.reshape(testbodymat1[:,0:2,:],(testbodymat1.shape[0],-1)).transpose(0,1)
refbodymat1 = np.reshape(refbodymat1[:,0:2,:],(refbodymat1.shape[0],-1)).transpose(0,1)

testbodymat2 = np.reshape(testbodymat2[:,0:2,:],(testbodymat2.shape[0],-1)).transpose(0,1)
refbodymat2 = np.reshape(refbodymat2[:,0:2,:],(refbodymat2.shape[0],-1)).transpose(0,1)

testbodymat3 = np.reshape(testbodymat3[:,0:2,:],(testbodymat3.shape[0],-1)).transpose(0,1)
refbodymat3 = np.reshape(refbodymat3[:,0:2,:],(refbodymat3.shape[0],-1)).transpose(0,1)

path, dist = tsl.dtw_path(testbodymat1[:,:], refbodymat1[:,:])
pathlist.append(path)
distlist.append(dist)

path, dist = tsl.dtw_path(testbodymat2[:,:], refbodymat2[:,:])
pathlist.append(path)
distlist.append(dist)

path, dist = tsl.dtw_path(testbodymat3[:,:], refbodymat3[:,:])
pathlist.append(path)
distlist.append(dist)

print(pathlist[0])
print(pathlist[1])
print(distlist)

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (9, 8), (10, 8), (11, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 9), (18, 9), (19, 10), (20, 10), (21, 10), (22, 11), (23, 11), (24, 12), (25, 12), (26, 12), (27, 13), (28, 13), (29, 14), (30, 14), (31, 14), (32, 15), (33, 16), (34, 17), (35, 18), (36, 19), (37, 20), (38, 21), (39, 22), (40, 23), (41, 24), (42, 25), (43, 25), (44, 25), (45, 25), (46, 25), (47, 25), (48, 25), (49, 25), (50, 26), (51, 26), (52, 26), (53, 27), (54, 28), (55, 29), (56, 30), (57, 31), (58, 32), (59, 33), (60, 34), (61, 35), (62, 36), (63, 37), (64, 38), (65, 39), (66, 40), (67, 41), (68, 42), (69, 43), (70, 44), (71, 45), (72, 46), (73, 46), (74, 47), (75, 48), (76, 49), (77, 50), (78, 51), (79, 52), (80, 53), (81, 54), (82, 55), (83, 55), (84, 55), (85, 55), (86, 55), (87, 55), (88, 55), (89, 55), (90, 55), (91, 55), (92, 55), (93, 55), (94, 56), (95, 57), (95, 58), (95, 59), 

In [ ]:
print(len(path))

2064


In [ ]:
import math
numkeypoints = 15
errorbody = np.zeros([testbodymat.shape[0],numkeypoints-1])
errorval = [20,20,20]

#for k in range(0,numkeypoints-1):
for k in range(0,numkeypoints-1):
  i0 = -1
  j0 = -1
  ilist = []
  jlist = []
  if k in [2,3,5,6]:
    k1 = 0
  elif k in [0,1,4,7,8,11]:
    k1 = 1
  elif k in [9,10,12,13]:
    k1 = 2
  for p in range(0,len(pathlist[k1])):
    i = pathlist[k1][p][0]
    j = pathlist[k1][p][1]

    if i==(i0+1):
      jlist.append(j)
    else:
      jlen = len(jlist)
      #print(jlist)
      jj = jlist[int(np.floor(jlen/2))]
      #print(jj)

      testvect = [testbodymat[i,2,k],testbodymat[i,3,k],testbodymat[i,4,k]]
      testmag = (testbodymat[i,2,k])**2 + (testbodymat[i,3,k])**2 + (testbodymat[i,4,k])**2
      refvect = [refbodymat[jj,2,k],refbodymat[jj,3,k],refbodymat[jj,4,k]]
      refmag = (refbodymat[jj,2,k])**2 + (refbodymat[jj,3,k])**2 + (refbodymat[jj,4,k])**2

      val = math.degrees(math.acos(np.dot(testvect,refvect)/np.sqrt(testmag)/np.sqrt(refmag)))
      #print(i)
      # print(testmag)
      # print(jj)
      # print(refmag)
      #print(val)
      if val > errorval[k1]:
        errorbody[i,k] = 1
        # if k ==7:
        #   print(testvect)
        #   print(refvect)
      
      jlist = []
      jlist.append(j)
      i0 = i-1


In [ ]:
for i in range(0,len(errorbody)):
  if errorbody[i][2]==1 or errorbody[i][3]==1:
    errorbody[i][2]=1
    errorbody[i][3]=1
  if errorbody[i][5]==1 or errorbody[i][6]==1:
    errorbody[i][5]=1
    errorbody[i][6]=1
  if errorbody[i][9]==1 or errorbody[i][10]==1:
    errorbody[i][9]=1
    errorbody[i][10]=1
  if errorbody[i][12]==1 or errorbody[i][13]==1:
    errorbody[i][12]=1
    errorbody[i][13]=1

In [ ]:
## individual joints
# errorbody = np.zeros([testbodymat.shape[0],numkeypoints-1])
# errorval = 50

# for k in range(0,numkeypoints-1):
#   i0 = -1
#   j0 = -1
#   ilist = []
#   jlist = []
#   for p in range(0,len(pathlist[k])):
#     i = pathlist[k][p][0]
#     j = pathlist[k][p][1]

#     if i==(i0+1):
#       jlist.append(j)
#     else:
#       jlen = len(jlist)
#       #print(jlen)
#       j = jlist[int(np.floor(jlen/2))]
#       #print(j)

#       testvect = [testbodymat[i,2,k],testbodymat[i,3,k],testbodymat[i,4,k]]
#       testmag = (testbodymat[i,2,k])**2 + (testbodymat[i,3,k])**2 + (testbodymat[i,4,k])**2
#       refvect = [refbodymat[j,2,k],refbodymat[j,3,k],refbodymat[j,4,k]]
#       refmag = (refbodymat[j,2,k])**2 + (refbodymat[j,3,k])**2 + (refbodymat[j,4,k])**2

#       val = math.degrees(math.acos(np.dot(testvect,refvect)/np.sqrt(testmag)/np.sqrt(refmag)))
#       if val > errorval:
#         errorbody[i,k] = 1
      
#       jlist = []
#       jlist.append(j)
#       i0 = i-1


In [ ]:
## whole body
# errorbody = np.zeros([testbodymat.shape[0],numkeypoints-1])
# errorval = 50


# for p in range(0,len(path)):
#   i = path[p][0]
#   j = path[p][1]
#   for k in range(0,numkeypoints-1):
#     testvect = [testbodymat[i,2,k],testbodymat[i,3,k],testbodymat[i,4,k]]
#     testmag = (testbodymat[i,2,k])**2 + (testbodymat[i,3,k])**2 + (testbodymat[i,4,k])**2
#     refvect = [refbodymat[j,2,k],refbodymat[j,3,k],refbodymat[j,4,k]]
#     refmag = (refbodymat[j,2,k])**2 + (refbodymat[j,3,k])**2 + (refbodymat[j,4,k])**2

#     val = math.degrees(math.acos(np.dot(testvect,refvect)/np.sqrt(testmag)/np.sqrt(refmag)))
#     if val > errorval:
#       errorbody[i,k] = 1

#     # angdiffelev = np.abs(bodymat[i][0][k]-refbodymat[j][0][k])
#     # if (angdiffelev>180):
#     #   angdiffelev = np.abs(angdiffelev - 360)
#     # angdiffazi = np.abs(bodymat[i][1][k]-refbodymat[j][1][k])
#     # if (angdiffazi>180):
#     #   angdiffazi = np.abs(angdiffazi - 360)
#     # if (angdiffelev > 45) or (angdiffazi > 45):
#     #   errorbody[i,k] = 1


In [ ]:
#errorbody[-3:-1] = errorbody[-5]
print(errorbody[1428])

[1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0.]


Obtain keypoint data

In [ ]:
folderlist0 = sorted(os.listdir("/content/expose/folder/keypoints"))
path0 = "/content/expose/folder/keypoints"
data0list = []
for j in range(0,len(folderlist0)):

  # Opening JSON file
  with open(os.path.join(path0,folderlist0[j])) as json_file0:
      data0 = json.load(json_file0)
  
  data0list.append(data0['people'][0]['pose_keypoints_2d'])

print(data0list[-1])

[482.639, 434.934, 0.910046, 513.903, 602.957, 0.937913, 393.331, 603.012, 0.903127, 356.884, 812.319, 0.90502, 387.942, 1016.57, 0.852299, 629.048, 602.861, 0.853203, 660.542, 801.88, 0.843129, 644.952, 985.395, 0.938822, 518.832, 980.077, 0.783586, 440.483, 980.051, 0.745314, 477.112, 1278.41, 0.8612, 503.224, 1524.72, 0.792358, 576.793, 985.108, 0.725462, 571.507, 1278.42, 0.871579, 566.28, 1524.62, 0.837704, 466.55, 403.822, 0.87033, 518.847, 398.611, 0.948575, 435.178, 435.48, 0.913433, 560.914, 419.557, 0.908724, 576.725, 1608.7, 0.718056, 608.348, 1598.14, 0.710803, 561.055, 1530.25, 0.68272, 487.75, 1608.6, 0.888012, 466.549, 1603.17, 0.840923, 513.919, 1540.46, 0.645468]


Export openpose keypoint for plotting

In [ ]:
np.save('vid_keypoint.npy', data0list)

In [ ]:
ref_file = "vid_keypoint.npy"
gdrive_path = "/content/gdrive/MyDrive/datasets"
src_path = "/content/expose"
shutil.copyfile(os.path.join(src_path, ref_file), os.path.join(gdrive_path, ref_file))

'/content/gdrive/MyDrive/datasets/vid_keypoint.npy'

In [ ]:
import pickle

In [ ]:
jointcoord = [[1,0],[1,2],[2,3],[3,4],[1,5],[5,6],[6,7],[8,1],[8,9],[9,10],[10,11],[8,12],[12,13],[13,14]]

output_dict = {'vid_keypoint':data0list, 'errorlist':errorbody, 'jointref':jointcoord}

filename = 'keypoint_output_full3_incorrect.pkl'
outfile = open(filename,'wb')
pickle.dump(output_dict,outfile)
outfile.close()

In [ ]:
ref_file = "keypoint_output_full3_incorrect.pkl"
gdrive_path = "/content/gdrive/MyDrive/datasets"
src_path = "/content/expose"
shutil.copyfile(os.path.join(src_path, ref_file), os.path.join(gdrive_path, ref_file))

'/content/gdrive/MyDrive/datasets/keypoint_output_full3_incorrect.pkl'

In [ ]:
ref_file = "keypoint_output_full.pkl"
gdrive_path = "/content/gdrive/MyDrive/datasets"
dst_path = "/content/expose"
shutil.copyfile(os.path.join(gdrive_path, ref_file), os.path.join(dst_path, ref_file))

'/content/expose/keypoint_output_full.pkl'

In [ ]:
infile = open("keypoint_output_full.pkl",'rb')
in_dict = pickle.load(infile)
infile.close()
print(in_dict)

{'vid_keypoint': [[519.24, 597.712, 0.923689, 524.5, 754.721, 0.884707, 419.57, 749.503, 0.812666, 393.056, 943.21, 0.918061, 398.43, 1121.47, 0.893854, 644.736, 749.547, 0.857567, 681.296, 937.941, 0.905592, 644.898, 1116.07, 0.928586, 524.499, 1110.79, 0.803969, 466.598, 1110.85, 0.736635, 487.68, 1393.59, 0.844786, 524.417, 1618.95, 0.849635, 602.658, 1110.77, 0.79407, 602.676, 1398.73, 0.876977, 597.475, 1629.51, 0.791653, 487.818, 571.383, 0.891065, 545.263, 566.328, 0.879824, 466.615, 602.688, 0.884107, 576.761, 592.338, 0.851222, 592.326, 1718.53, 0.776542, 618.676, 1702.75, 0.800477, 587.065, 1650.55, 0.611032, 519.254, 1723.7, 0.767053, 482.635, 1702.8, 0.775448, 539.911, 1645.21, 0.629632], [519.265, 597.678, 0.925482, 524.522, 754.664, 0.882749, 424.644, 744.401, 0.817675, 388.081, 938.222, 0.914594, 393.459, 1116.32, 0.906367, 644.721, 749.58, 0.858113, 681.496, 937.867, 0.909527, 644.864, 1110.94, 0.949056, 524.401, 1105.78, 0.808071, 461.384, 1105.84, 0.731887, 487.698, 1

In [ ]:
data0list = in_dict['vid_keypoint']
#errorbody = in_dict['errorlist']
jointcoord = in_dict['jointref']

Plot keypoints and links

# Plot output

In [ ]:
!pip install matplotlib==3.1.3

     |████████████████████████████████| 13.1 MB 4.3 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from matplotlib import pyplot as plt

%cd /content/expose

!rm -rf /content/expose/result
!mkdir /content/expose/result

imglist = sorted(os.listdir("/content/expose/folder/images"))
imgpath = "/content/expose/folder/images"

jointcoord = [[1,0],[1,2],[2,3],[3,4],[1,5],[5,6],[6,7],[8,1],[8,9],[9,10],[10,11],[8,12],[12,13],[13,14]]


for i,im in enumerate(imglist):
  if 1 == 1:
    image = cv2.imread(os.path.join(imgpath,imglist[i]))
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    for j in range(0,numkeypoints):
      image = cv2.circle(image, (int(data0list[i][(j+1)*3-3]),int(data0list[i][(j+1)*3-2])), radius=8, color=(0, 0, 255), thickness=-1)
    for k in range(0,len(errorbody[i])):
      x1 = int(data0list[i][(jointcoord[k][0]+1)*3-3])
      y1 = int(data0list[i][(jointcoord[k][0]+1)*3-2])
      x2 = int(data0list[i][(jointcoord[k][1]+1)*3-3])
      y2 = int(data0list[i][(jointcoord[k][1]+1)*3-2])

      histtotal = 0
      for hist in range(0,10):
        histtotal += errorbody[i-hist][k]

      if i>9 and histtotal > 4:
        image = cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), thickness=8)
      else:
        image = cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), thickness=8)
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    save_impath = "/content/expose/result/result_frame_"+str(i).zfill(10)+".jpg"
    cv2.imwrite(save_impath,image)
    # plt.figure(figsize = (100,10))
    # plt.imshow(image)
    # plt.show()

cv2.destroyAllWindows()


/content/expose


Export video output with plotted keypoints and links

In [ ]:
!rm -rf final.mp4

resultlist = sorted(os.listdir("/content/expose/result"))
resultpath = "/content/expose/result"
resultimg=[]

for i in range(0,len(resultlist)):
  image = cv2.imread(os.path.join(resultpath,resultlist[i]))
  resultimg.append(image)


height,width,layers=resultimg[1].shape

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video=cv2.VideoWriter('final.mp4',fourcc,30,(width,height))

for j in range(0,len(resultlist)):
    video.write(resultimg[j])

cv2.destroyAllWindows()
video.release()